In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv")

In [3]:
df["card"] = (df["card"]=="yes").astype('int8')
df["owner"] = (df["owner"]=="yes").astype('int8')
df["selfemp"] = (df["selfemp"]=="yes").astype('int8')

In [4]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train["card"].values
y_val = df_val["card"].values
y_test = df_test["card"].values

del df_train["card"]
del df_val["card"]
del df_test["card"]

In [6]:
from sklearn.metrics import roc_auc_score

In [7]:
features = ["reports", "age", "income", "share", "expenditure",
 "dependents", "months", "majorcards", "active", "owner", "selfemp"]

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
dv = DictVectorizer(sparse=False)

In [9]:
train_dict = df_train[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

val_dicts = df_val[features].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_pred = model.predict_proba(X_val)[:, 1]

In [10]:
from sklearn.model_selection import KFold

In [11]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[features].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

def predict(df, dv, model):
    dicts = df[features].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [13]:
C = 1.0
n_splits = 5

In [14]:
scores = []

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.card.values
    y_val = df_val.card.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.996 +- 0.003


In [15]:
import pickle

In [17]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [19]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [20]:
import pickle
model_file = 'model_C=1.0.bin'

In [21]:
with open(output_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [22]:
dv, model

(DictVectorizer(sparse=False),
 LogisticRegression(max_iter=1000, solver='liblinear'))

In [40]:
customer = {'reports': 0.0,
 'age': 60.83333,
 'income': 2.5,
 'share': 0.4028644,
 'expenditure': 8390.3008,
 'owner': 0.0,
 'selfemp': 0.0,
 'dependents': 0.0,
 'months': 4.0,
 'majorcards': 1.0,
 'active': 1.0}

In [41]:
X = dv.transform([customer])

In [42]:
y_pred = model.predict_proba(X)[0, 1]

1.0

In [ ]:
print('input:', customer)
print('output:', y_pred)